In [ ]:
import torch

from utils import (
    get_model,
    get_data,
)

In [ ]:
train_dl, val_dl = get_data()

# High-level training libraries

Writing the training loop is my least favourite thing about PyTorch.

Keras is great here!
```python
model.compile(optimizer, criterion, metrics=["accuracy", "f1"])
model.fit(X, y, epochs=10)
```


### [Ignite](https://github.com/pytorch/ignite)
> Ignite is a high-level library to help with training neural networks in PyTorch.
> - ignite helps you write compact but full-featured training loops in a few lines of code
> - you get a training loop with metrics, early-stopping, model checkpointing and other features without the boilerplate


### [TNT](https://github.com/pytorch/tnt)
> TNT is a library providing powerful dataloading, logging and visualization utlities for Python. It is closely intergrated with PyTorch and is designed to enable rapid iteration with any model or training regimen.
> [...]
> The project was inspired by TorchNet, and legend says that it stood for “TorchNetTwo”


### [Skorch](https://github.com/dnouri/skorch)
> A scikit-learn compatible neural network library that wraps PyTorch.


### [PyToune](https://pytoune.org/)
> PyToune is a Keras-like framework for PyTorch and handles much of the boilerplating code needed to train neural networks.


### [fast.ai](https://docs.fast.ai/)
> The fastai library simplifies training fast and accurate neural nets using modern best practices. It's based on research in to deep learning best practices undertaken at fast.ai, including "out of the box" support for vision, text, tabular, and collab (collaborative filtering) models.


### [MagNet](https://github.com/MagNet-DL/magnet)
> MagNet is a high-level Deep Learning API, wrapped around PyTorch.
> 
> It was developed with the aim of reducing boilerplate code and writing Deep Learning architectures with more grace.

```python
model = nn.Sequential(mn.Conv(32), *mn.Conv() * 2, mn.Linear(10, act=None))
```

### "The fun of Reinvention"
Clearly, there must be a better way! Write your own lib (but don't use it) :D

## Demo with Ignite

In [ ]:
import ignite
from ignite.metrics import (
    CategoricalAccuracy,
    Loss,
    Precision,
)
from ignite.engine import (
    create_supervised_evaluator,
    create_supervised_trainer,
    Events,
)

In [ ]:
# model, loss, optimizer
model = get_model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    get_trainable(model.parameters()),
    lr=0.001,
    momentum=.9,
)

# trainer and evaluator
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(
    model,
    metrics={
        "accuracy": CategoricalAccuracy(),
        "loss": Loss(criterion),
        "precision": Precision(),
    },
    device=device,
)

In [ ]:
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    print(f"Epoch[{engine.state.epoch}] Batch[{engine.state.iteration}] Loss: {engine.state.output:.2f}")


# trainer.run(train_dl, max_epochs=1)

In [ ]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_dl)
    metrics = evaluator.state.metrics
    print(f"Training Results   - Epoch: {trainer.state.epoch}  "
          f"accuracy: {metrics['accuracy']:.2f} "
          f"loss: {metrics['loss']:.2f} "
          f"prec: {metrics['precision'].cpu()}")


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(val_dl)
    metrics = evaluator.state.metrics
    print(f"Validation Results - Epoch: {trainer.state.epoch}  "
          f"accuracy: {metrics['accuracy']:.2f} "
          f"loss: {metrics['loss']:.2f} "
          f"prec: {metrics['precision'].cpu()}")


trainer.run(train_dl, max_epochs=1)

# Visualization with Tensorboard
- https://www.tensorflow.org/programmers_guide/summaries_and_tensorboard
- https://github.com/lanpa/tensorboard-pytorch

Demo: https://github.com/lanpa/tensorboard-pytorch/blob/master/screenshots/Demo.gif


Start tensorboard:
```
cd notebooks
tensorboard --logdir=tf_log
```

In [ ]:
rm -r tf_log/*

In [ ]:
ls tf_log/

In [ ]:
from tensorboardX import SummaryWriter

summary_writer = SummaryWriter(log_dir=f"tf_log/exp_{random.randint(0, 100)}")

In [ ]:
ls tf_log

In [ ]:
# write some scalars
for i in range(10):
    summary_writer.add_scalar("training/loss", np.random.rand(), i)
    summary_writer.add_scalar("validation/loss", np.random.rand() + .1, i)

Then visit http://localhost:6006

In [ ]:
# Visualize the graph/network
X, _ = next(iter(train_dl))
summary_writer.add_graph(model, X)

## Use tensorboard with ignite

In [ ]:
# new SummaryWriter for new experiment
now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
summary_writer = SummaryWriter(log_dir=f"tf_log/exp_ignite_{now}")

# Basic setup: model, loss, optimizer
model = get_model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(get_trainable(model.parameters()), lr=0.0001, momentum=.9)

# trainer and evaluator
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(
    model,
    metrics={"accuracy": CategoricalAccuracy(), "loss": Loss(criterion)},
    device=device,
)

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_dl)
    metrics = evaluator.state.metrics
    epoch = trainer.state.epoch
    summary_writer.add_scalar("training/accuracy", metrics['accuracy'], epoch)
    summary_writer.add_scalar("training/loss", metrics['loss'], epoch)


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(val_dl)
    metrics = evaluator.state.metrics
    epoch = trainer.state.epoch
    summary_writer.add_scalar("validation/accuracy", metrics['accuracy'], epoch)
    summary_writer.add_scalar("validation/loss", metrics['loss'], epoch)
    print(metrics['accuracy'])


trainer.run(train_dl, 15)

## Visdom
https://github.com/facebookresearch/visdom
![](https://camo.githubusercontent.com/d69475a01f9f327fc42931a21df8134d1fbdfc19/68747470733a2f2f6c68332e676f6f676c6575736572636f6e74656e742e636f6d2f2d62714839555843772d42452f574c3255736472726241492f41414141414141416e59632f656d727877436d6e7257345f434c54797955747442305359524a2d693443436951434c63422f73302f53637265656e2b53686f742b323031372d30332d30362b61742b31302e35312e30322b414d2e706e67253232766973646f6d5f626967253232)